In [1]:
import pandas as pd 

pd.read_csv("train.csv")

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
1816,Train_1816,0,0,0,1
1817,Train_1817,1,0,0,0
1818,Train_1818,1,0,0,0
1819,Train_1819,0,0,1,0


In [2]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

# 1. 라벨 전처리
df = pd.read_csv('train.csv')
df['label'] = df[['healthy', 'multiple_diseases', 'rust', 'scab']].idxmax(axis=1)
label_map = {'healthy': 0, 'multiple_diseases': 1, 'rust': 2, 'scab': 3}
df['label'] = df['label'].map(label_map)
df

c:\Users\admin\anaconda3\envs\torchenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,image_id,healthy,multiple_diseases,rust,scab,label
0,Train_0,0,0,0,1,3
1,Train_1,0,1,0,0,1
2,Train_2,1,0,0,0,0
3,Train_3,0,0,1,0,2
4,Train_4,1,0,0,0,0
...,...,...,...,...,...,...
1816,Train_1816,0,0,0,1,3
1817,Train_1817,1,0,0,0,0
1818,Train_1818,1,0,0,0,0
1819,Train_1819,0,0,1,0,2


In [3]:

# 2. 데이터셋 클래스 정의
class PlantDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'image_id'].lower() + '.jpg'
        label = self.df.loc[idx, 'label']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# 3. 데이터 분할
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [42]:
# 4. 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])
from torch.utils.data import WeightedRandomSampler
import numpy as np

# 각 클래스별 개수 확인
label_counts = train_df['label'].value_counts().to_dict()

# 클래스별 가중치 계산 (데이터 수가 적을수록 높은 weight)
weights = [1.0 / label_counts[label] for label in train_df['label']]


train_dataset = PlantDataset(train_df, './images', transform)
val_dataset = PlantDataset(val_df, './images', transform)

# 클래스 불균형 보완용 Sampler
label_counts = train_df['label'].value_counts().to_dict()
weights = [1.0 / label_counts[label] for label in train_df['label']]
sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

# 데이터로더 정의
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [43]:
# 6. 모델 정의 (ResNeXt50)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 클래스 수: 4
model = model.to(device)

# 7. 손실함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(model.parameters(), lr=1e-4)

In [44]:
from tqdm import tqdm
import torch

# 🔸 EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_acc):
        if self.best_score is None:
            self.best_score = val_acc
        elif val_acc < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_acc
            self.counter = 0

# 🔹 학습 루프
num_epochs = 50  # EarlyStopping이 있어서 길게 잡아도 됨
early_stopping = EarlyStopping(patience=3)  # 3번 연속 성능 안 오르면 종료

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    print(f"\n🟢 Epoch {epoch + 1}/{num_epochs} - Training")
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"✅ Epoch [{epoch+1}/{num_epochs}] | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")

    # 🔵 검증
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0

    print(f"🔵 Epoch {epoch + 1}/{num_epochs} - Validation")
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    print(f"📊 Validation Loss: {val_loss:.4f} | Accuracy: {val_acc:.2f}%\n")

    # 🛑 EarlyStopping 체크
    early_stopping(val_acc)
    if early_stopping.early_stop:
        print("⛔ Early stopping triggered. Training stopped.")
        break



🟢 Epoch 1/50 - Training


✅ Epoch [1/50] | Loss: 13.8499 | Accuracy: 78.91%
🔵 Epoch 1/50 - Validation


📊 Validation Loss: 1.4145 | Accuracy: 92.05%


🟢 Epoch 2/50 - Training


✅ Epoch [2/50] | Loss: 2.5884 | Accuracy: 96.50%
🔵 Epoch 2/50 - Validation


📊 Validation Loss: 1.0512 | Accuracy: 94.79%


🟢 Epoch 3/50 - Training


...

📊 Validation Loss: 1.6426 | Accuracy: 92.88%


🟢 Epoch 4/50 - Training


✅ Epoch [4/50] | Loss: 1.2511 | Accuracy: 98.21%
🔵 Epoch 4/50 - Validation


📊 Validation Loss: 1.5141 | Accuracy: 93.97%


🟢 Epoch 5/50 - Training


✅ Epoch [5/50] | Loss: 0.8498 | Accuracy: 98.76%
🔵 Epoch 5/50 - Validation


📊 Validation Loss: 1.2320 | Accuracy: 93.70%

⛔ Early stopping triggered. Training stopped.


In [45]:
torch.save(model.state_dict(), 'resnext_model.pth')
print("📁 모델 저장 완료: resnext_model.pth")


📁 모델 저장 완료: resnext_model.pth


In [86]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

#device 변수에 torch에 있는 함수인 device를 불러와 cuda를 사용할건데 만약 cuda가 안돼면 cpu로
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#이미지를 전처리하는 작업
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

test_img_path = 2
#
img_path = f"C:/Users/admin/Desktop/oshLecture/Plant Pathology 2020 - FGVC7/images/Test_{test_img_path}.jpg" 
#img_path = "C:/Users/admin/Desktop/oshLecture/Plant Pathology 2020 - FGVC7/images/Train_1794.jpg"
image = Image.open(img_path).convert('RGB')


#
image = transform(image).to(device)

In [72]:
# resnext라는 모델(알로리즘/틀)을 불러옴
model = models.resnext50_32x4d(pretrained=True)
#출력층 4로 저장
model.fc = nn.Linear(model.fc.in_features, 4) 
#torch에 있는 이미 학습이 된 resnext_model.pth안에 가중치를 불러와서 state_dict변수에 저장
state_dict = torch.load('resnext_model.pth')
#가중치 전이
model.load_state_dict(state_dict)
#GPU(cuda)위에 모델을 실행하겠다고 선언
model = model.to(device)


In [87]:
#이미지를 모델에 넣어서 예측하는데 확률값이 나옴 .view인자에서 첫번째 인자는 이미지 배치 차원 추가, # [데이터갯수, 채널, 가로, 세로]
class_prob= model(image.view(1,3, 224, 224))
print(class_prob)
#많은 확률값중 가장 높은 확률만 골라 출력, 인덱스로 예측값
result = torch.argmax(class_prob)
result


tensor([[-3.2813, -2.1035, -3.7011,  7.4729]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


tensor(3, device='cuda:0')

In [88]:
# 인덱스를 직관적으로 보기위해 리스트 선언
label_map = ['healthy', 'multiple_diseases', 'rust', 'scab']
#출력
label_map[result]

'scab'

In [82]:
from collections import Counter

model.eval()
all_preds = []
with torch.no_grad():
    for images, _ in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())

print(Counter(all_preds))

Counter({2: 124, 3: 122, 0: 104, 1: 15})
